In [0]:
import re
import pyspark.sql.types
from pyspark.sql import functions as fc


file_path = "/Volumes/workspace/default/data-volume/1970-2021_DISASTERS.xlsx - emdat data.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)


In [0]:
display(df.limit(10))

df.printSchema()

# Colunas que devem ser observadas: Dis No, Seq, ISO, Associated Dis, Associated Dis2, Dis Mag Value, Dis Mag Scale, Reconstruction Costs ('000 US$), Insured Damages ('000 US$),
# #Total Damages ('000 US$)

In [0]:
def clean_column(col_name):
    col_name = col_name.lower()                                 #transforma em minusculas
    col_name = col_name.replace(" ", "_").replace("-", "_")     #remove espaços e traços por underscore                                    
    col_name = re.sub(r"[ ,;{}()\n\t=]", "_", col_name)         # troca caracteres problemáticos por _
    col_name = re.sub(r"[^0-9a-zA-Z_]", "", col_name)           # remove qualquer outro caractere especial
    col_name = re.sub(r"_+", "_", col_name)                     # colapsa underscores duplos
    col_name = col_name.strip("_")                              # remove _ no início/fim
    
    return col_name

#aplica a função em todas as colunas do dataframe, percorrendo com o loop for
df = df.toDF(*[clean_column(c) for c in df.columns])

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("bronze_disasters")

df.display()

In [0]:
%sql

DESCRIBE DETAIL `bronze_disasters`


In [0]:
df = spark.table("bronze_disasters")